In [3]:
import os

def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add




In [4]:
files_add=find_the_way("./","data.csv")
files_add=[ './ValidationGeo\\data.csv',
 './ValidationWDE\\data.csv','./TrainingGeo\\data.csv',
 './TrainingWDE\\data.csv']

In [5]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import json
import numpy as np
from collections import defaultdict, Counter
import os
from pathlib import Path

# Bina-Kat sözlüğü
BF_DICTIONARY = {
    '00': 0, '01': 1, '02': 2, '03': 3,
    '10': 4, '11': 5, '12': 6, '13': 7,
    '20': 8, '21': 9, '22': 10, '23': 11, '24': 12
}

def get_bf_key(building_id, floor_id):
    """Bina ve kat ID'lerinden benzersiz anahtar oluşturur."""
    return f"{int(building_id)}{int(floor_id)}"

def analyze_data_distribution(df):
    """Veri dağılımını analiz eder."""
    print("\n📊 VERİ DAĞILIMI ANALİZİ:")
    print("=" * 70)
    
    for building in sorted(df['BUILDINGID'].unique()):
        building_df = df[df['BUILDINGID'] == building]
        print(f"\n🏢 Bina {int(building)}:")
        
        for floor in sorted(building_df['FLOOR'].unique()):
            floor_df = building_df[building_df['FLOOR'] == floor]
            bf_key = get_bf_key(building, floor)
            class_id = BF_DICTIONARY.get(bf_key, -1)
            
            print(f"   Kat {int(floor)} (Class {class_id}): {len(floor_df)} fingerprint")
    
    # Class dağılımı
    print("\n🎯 SINIF DAĞILIMI:")
    class_counts = Counter()
    for _, row in df.iterrows():
        bf_key = get_bf_key(row['BUILDINGID'], row['FLOOR'])
        class_id = BF_DICTIONARY.get(bf_key, -1)
        class_counts[class_id] += 1
    
    for class_id in sorted(class_counts.keys()):
        print(f"   Class {class_id}: {class_counts[class_id]} fingerprint")
    
    # Dengesizlik oranı
    if len(class_counts) > 0:
        max_samples = max(class_counts.values())
        min_samples = min(class_counts.values())
        print(f"\n⚠️ Dengesizlik Oranı: {max_samples/min_samples:.2f}x")
        if max_samples/min_samples > 3:
            print("   → Veri dengesizliği yüksek! Bu skorları düşürüyor olabilir.")

def convert_uji_to_task2(input_csv, output_prefix, 
                         time_threshold=300,  # 5 dakika (artırıldı)
                         use_space_id=False):  # SPACEID'yi trajectory için kullanma
    """
    UJIIndoorLoc formatını Task 2 formatına dönüştürür.
    
    Args:
        input_csv: UJIIndoorLoc CSV dosyası
        output_prefix: Çıktı dosyaları için prefix
        time_threshold: Trajectory ayırma için zaman eşiği (saniye)
        use_space_id: SPACEID'ye göre trajectory ayırma (False önerilir)
    """
    
    print("=" * 70)
    print(f"📍 UJIIndoorLoc -> Task2 Format Dönüşümü")
    print(f"📂 Girdi: {input_csv}")
    print(f"⏱️ Zaman Eşiği: {time_threshold} saniye")
    print(f"🏠 SPACEID Kullanımı: {use_space_id}")
    print("=" * 70)
    
    # Veriyi yükle
    df = pd.read_csv(input_csv)
    print(f"\n📊 Toplam fingerprint: {len(df)}")
    print(f"📊 Bina sayısı: {df['BUILDINGID'].nunique()}")
    print(f"📊 Kat sayısı: {df['FLOOR'].nunique()}")
    print(f"📊 Kullanıcı sayısı: {df['USERID'].nunique()}")
    
    # Veri dağılımını analiz et
    analyze_data_distribution(df)
    
    # WAP kolonlarını bul
    wap_columns = [col for col in df.columns if col.startswith('WAP')]
    print(f"\n📊 WAP sayısı: {len(wap_columns)}")
    
    # ============================================================
    # 1. VERİYİ TEMIZLE VE SIRALA
    # ============================================================
    print("\n🔄 Veri hazırlanıyor...")
    
    df['original_idx'] = df.index
    
    # Önce kullanıcı/telefon/bina/kat'a göre sırala, sonra zamana göre
    df = df.sort_values([
        'USERID', 'PHONEID', 'BUILDINGID', 'FLOOR', 'TIMESTAMP'
    ]).reset_index(drop=True)
    
    # ============================================================
    # 2. İYİLEŞTİRİLMİŞ TRAJECTORY OLUŞTURMA
    # ============================================================
    print("\n🛤️ Trajectory'ler oluşturuluyor (İyileştirilmiş Mantık)...")
    
    trajectory_id = 0
    trajectory_lookup = {}
    trajectory_metadata = defaultdict(lambda: {
        'fingerprints': [],
        'user': None,
        'phone': None,
        'building': None,
        'floor': None,
        'start_time': None,
        'end_time': None
    })
    
    prev_row = None
    
    for idx, row in df.iterrows():
        orig_idx = int(row['original_idx'])
        
        # Yeni trajectory başlatma koşulları (DAHA ESNEK)
        start_new = False
        
        if prev_row is None:
            start_new = True
        else:
            # KRİTİK: Kullanıcı/telefon değişimi
            user_changed = (row['USERID'] != prev_row['USERID'] or 
                          row['PHONEID'] != prev_row['PHONEID'])
            
            # KRİTİK: Bina veya KAT değişimi (kesinlikle yeni trajectory)
            location_changed = (row['BUILDINGID'] != prev_row['BUILDINGID'] or 
                              row['FLOOR'] != prev_row['FLOOR'])
            
            # Zaman farkı çok büyük
            time_gap = (row['TIMESTAMP'] - prev_row['TIMESTAMP']) > time_threshold
            
            # SPACEID kullanımı opsiyonel (genelde kullanılmamalı)
            space_changed = use_space_id and (row['SPACEID'] != prev_row['SPACEID'])
            
            # Trajectory'yi sadece bu durumlarda kes
            start_new = user_changed or location_changed or time_gap or space_changed
        
        if start_new:
            trajectory_id += 1
            trajectory_metadata[trajectory_id].update({
                'user': row['USERID'],
                'phone': row['PHONEID'],
                'building': int(row['BUILDINGID']),
                'floor': int(row['FLOOR']),
                'start_time': row['TIMESTAMP']
            })
        
        # Trajectory'ye fingerprint ekle
        trajectory_lookup[str(orig_idx)] = trajectory_id
        trajectory_metadata[trajectory_id]['fingerprints'].append(orig_idx)
        trajectory_metadata[trajectory_id]['end_time'] = row['TIMESTAMP']
        
        prev_row = row
    
    # İstatistikler
    traj_sizes = [len(meta['fingerprints']) for meta in trajectory_metadata.values()]
    traj_durations = [(meta['end_time'] - meta['start_time']) 
                     for meta in trajectory_metadata.values() if meta['end_time'] and meta['start_time']]
    
    print(f"✅ {len(trajectory_metadata)} trajectory oluşturuldu")
    print(f"   • Min fingerprint/trajectory: {min(traj_sizes)}")
    print(f"   • Max fingerprint/trajectory: {max(traj_sizes)}")
    print(f"   • Ortalama: {np.mean(traj_sizes):.1f}")
    print(f"   • Median: {np.median(traj_sizes):.0f}")
    if traj_durations:
        print(f"   • Ortalama süre: {np.mean(traj_durations):.1f} saniye")
    
    # Trajectory başına class dağılımı
    traj_classes = defaultdict(int)
    for traj_id, meta in trajectory_metadata.items():
        bf_key = get_bf_key(meta['building'], meta['floor'])
        class_id = BF_DICTIONARY.get(bf_key, -1)
        traj_classes[class_id] += 1
    
    print(f"\n📊 TRAJECTORY CLASS DAĞILIMI:")
    for class_id in sorted(traj_classes.keys()):
        print(f"   Class {class_id}: {traj_classes[class_id]} trajectory")
    
    # Dengesizlik uyarısı
    if len(traj_classes) > 0:
        max_traj = max(traj_classes.values())
        min_traj = min(traj_classes.values())
        if max_traj / min_traj > 3:
            print(f"\n⚠️ UYARI: Trajectory dağılımı dengesiz ({max_traj/min_traj:.1f}x)")
            print("   → Bu, bazı sınıfların öğrenilmesini zorlaştırabilir!")
    
    # ============================================================
    # 3. FINGERPRINTS JSON
    # ============================================================
    print("\n💾 Fingerprint'ler kaydediliyor...")
    
    fingerprints = []
    for idx, row in df.iterrows():
        orig_idx = int(row['original_idx'])
        
        # WiFi verilerini topla (100 = sinyal yok)
        wifi_data = {}
        valid_signals = 0
        signal_sum = 0
        
        for wap in wap_columns:
            rssi = row[wap]
            if rssi != 100:
                wifi_data[wap] = int(rssi)
                valid_signals += 1
                signal_sum += rssi
        
        fingerprints.append({
            'fingerprint_id': orig_idx,
            'wifi_data': wifi_data
        })
    
    output_file = f'{output_prefix}_fingerprints.json'
    with open(output_file, 'w') as f:
        json.dump(fingerprints, f, indent=2)
    
    # WiFi istatistikleri
    wifi_counts = [len(fp['wifi_data']) for fp in fingerprints]
    print(f"✅ {output_file} kaydedildi ({len(fingerprints)} fingerprint)")
    print(f"   • Ortalama WAP/fingerprint: {np.mean(wifi_counts):.1f}")
    print(f"   • Min WAP: {min(wifi_counts)}, Max WAP: {max(wifi_counts)}")
    
    if np.mean(wifi_counts) < 50:
        print(f"\n⚠️ UYARI: Ortalama WAP sayısı düşük!")
        print("   → WiFi sinyali zayıf fingerprint'ler öğrenmeyi zorlaştırır.")
    
    # ============================================================
    # 4. LOOKUP JSON
    # ============================================================
    print("\n💾 Lookup tablosu kaydediliyor...")
    
    output_file = f'{output_prefix}_lookup.json'
    with open(output_file, 'w') as f:
        json.dump(trajectory_lookup, f, indent=2)
    print(f"✅ {output_file} kaydedildi")
    
    # ============================================================
    # 5. İYİLEŞTİRİLMİŞ STEPS CSV
    # ============================================================
    print("\n💾 Steps oluşturuluyor (AYNI KAT içi bağlantılar)...")
    
    steps = []
    zero_distance_count = 0
    
    for traj_id, meta in trajectory_metadata.items():
        fps = meta['fingerprints']
        
        if len(fps) < 2:
            continue
        
        # Timestamp'e göre sırala
        fps_with_ts = [(fp, df[df['original_idx'] == fp]['TIMESTAMP'].iloc[0]) 
                       for fp in fps]
        fps_sorted = [fp for fp, _ in sorted(fps_with_ts, key=lambda x: x[1])]
        
        # Ardışık fingerprint'ler arasında bağlantı oluştur
        for i in range(len(fps_sorted) - 1):
            fp1 = fps_sorted[i]
            fp2 = fps_sorted[i + 1]
            
            # Konum bilgilerini al
            row1 = df[df['original_idx'] == fp1].iloc[0]
            row2 = df[df['original_idx'] == fp2].iloc[0]
            
            # KAT KONTROLÜ - Steps sadece aynı kattaki bağlantılar için
            if row1['FLOOR'] != row2['FLOOR']:
                continue  # Farklı katlar - elevation'a eklenecek
            
            # Euclidean mesafe
            distance = np.sqrt(
                (row1['LONGITUDE'] - row2['LONGITUDE'])**2 + 
                (row1['LATITUDE'] - row2['LATITUDE'])**2
            )
            
            if distance < 0.0001:
                zero_distance_count += 1
            
            steps.append({
                'id1': int(fp1),
                'id2': int(fp2),
                'displacement': float(distance)
            })
    
    output_file = f'{output_prefix}_steps.csv'
    pd.DataFrame(steps).to_csv(output_file, index=False)
    
    if steps:
        distances = [s['displacement'] for s in steps]
        print(f"✅ {output_file} kaydedildi ({len(steps)} step)")
        print(f"   • Min mesafe: {min(distances):.4f}")
        print(f"   • Max mesafe: {max(distances):.4f}")
        print(f"   • Ortalama: {np.mean(distances):.4f}")
        print(f"   • Median: {np.median(distances):.4f}")
        
        if zero_distance_count > 0:
            print(f"   • Sıfır mesafeli: {zero_distance_count} ({100*zero_distance_count/len(steps):.1f}%)")
    else:
        print(f"⚠️ Hiç step oluşturulamadı!")
    
    # ============================================================
    # 6. İYİLEŞTİRİLMİŞ ELEVATIONS CSV
    # ============================================================
    print("\n💾 Elevations oluşturuluyor (KAT DEĞİŞİMLERİ)...")
    
    elevations = []
    
    # Yöntem 1: Aynı trajectory içindeki kat değişimleri
    print("   → Trajectory içi kat değişimleri tespit ediliyor...")
    intra_traj_elevations = 0
    
    for traj_id, meta in trajectory_metadata.items():
        fps = meta['fingerprints']
        
        if len(fps) < 2:
            continue
        
        # Timestamp'e göre sırala
        fps_with_ts = [(fp, df[df['original_idx'] == fp]['TIMESTAMP'].iloc[0]) 
                       for fp in fps]
        fps_sorted = [fp for fp, _ in sorted(fps_with_ts, key=lambda x: x[1])]
        
        # Ardışık fingerprint'ler arasında KAT DEĞİŞİMİ kontrolü
        for i in range(len(fps_sorted) - 1):
            fp1 = fps_sorted[i]
            fp2 = fps_sorted[i + 1]
            
            row1 = df[df['original_idx'] == fp1].iloc[0]
            row2 = df[df['original_idx'] == fp2].iloc[0]
            
            # KAT DEĞİŞTİ mi?
            if row1['FLOOR'] != row2['FLOOR']:
                elevations.append({
                    'id1': int(fp1),
                    'id2': int(fp2)
                })
                intra_traj_elevations += 1
    
    print(f"   → {intra_traj_elevations} adet trajectory-içi kat değişimi bulundu")
    
    # Yöntem 2: Farklı katlar arası çapraz bağlantılar (daha fazla elevation için)
    print("   → Katlar arası çapraz bağlantılar ekleniyor...")
    inter_floor_elevations = 0
    
    for building in df['BUILDINGID'].unique():
        building_trajs = {
            tid: meta for tid, meta in trajectory_metadata.items()
            if meta['building'] == int(building)
        }
        
        # Kat bazında grupla
        floor_trajs = defaultdict(list)
        for tid, meta in building_trajs.items():
            floor_trajs[meta['floor']].append(tid)
        
        # Her kat çifti için elevation oluştur (daha dengeli)
        floors = sorted(floor_trajs.keys())
        for i, floor1 in enumerate(floors):
            for floor2 in floors[i+1:]:
                # Her kattan en fazla 5 trajectory seç (dengelemek için)
                trajs1 = floor_trajs[floor1][:5]
                trajs2 = floor_trajs[floor2][:5]
                
                # Kombinasyonlar oluştur
                for traj1 in trajs1:
                    for traj2 in trajs2:
                        # Her trajectory'den orta fingerprint'i seç (daha temsili)
                        fps1 = trajectory_metadata[traj1]['fingerprints']
                        fps2 = trajectory_metadata[traj2]['fingerprints']
                        
                        fp1 = fps1[len(fps1)//2]  # Ortadaki
                        fp2 = fps2[len(fps2)//2]  # Ortadaki
                        
                        elevations.append({
                            'id1': int(fp1),
                            'id2': int(fp2)
                        })
                        inter_floor_elevations += 1
    
    print(f"   → {inter_floor_elevations} adet katlar-arası bağlantı eklendi")
    
    # Tekrarları kaldır
    elevations_df = pd.DataFrame(elevations).drop_duplicates()
    output_file = f'{output_prefix}_elevations.csv'
    elevations_df.to_csv(output_file, index=False)
    print(f"✅ {output_file} kaydedildi ({len(elevations_df)} elevation)")
    
    if len(elevations_df) == 0:
        print(f"⚠️ UYARI: Hiç elevation bulunamadı!")
        print("   → Bu, modelin katlar arası ilişkileri öğrenmesini engelleyecektir!")
    
    # ============================================================
    # 7. GROUND TRUTH JSON
    # ============================================================
    print("\n💾 Ground truth kaydediliyor...")
    
    ground_truth = {}
    missing_keys = []
    
    for traj_id, meta in trajectory_metadata.items():
        bf_key = get_bf_key(meta['building'], meta['floor'])
        
        if bf_key in BF_DICTIONARY:
            ground_truth[str(traj_id)] = BF_DICTIONARY[bf_key]
        else:
            missing_keys.append(bf_key)
            ground_truth[str(traj_id)] = -1
    
    if missing_keys:
        print(f"⚠️ UYARI: {len(missing_keys)} adet bina-kat kombinasyonu sözlükte yok:")
        for key in set(missing_keys):
            print(f"   → {key}")
    
    output_file = f'{output_prefix}_GT.json'
    with open(output_file, 'w') as f:
        json.dump(ground_truth, f, indent=2)
    print(f"✅ {output_file} kaydedildi ({len(ground_truth)} trajectory)")
    
    # ============================================================
    # 8. DETAYLI İSTATİSTİKLER VE ÖNERİLER
    # ============================================================
    print("\n" + "=" * 70)
    print("📊 DETAYLI İSTATİSTİKLER VE ÖNERİLER")
    print("=" * 70)
    
    # Steps vs Elevations oranı
    if steps and elevations_df is not None and len(elevations_df) > 0:
        ratio = len(steps) / len(elevations_df)
        print(f"\n📈 Steps/Elevations Oranı: {ratio:.2f}")
        if ratio < 5:
            print("   ⚠️ Elevation sayısı çok yüksek! Steps artırılmalı.")
        elif ratio > 50:
            print("   ⚠️ Elevation sayısı çok düşük! Katlar arası bağlantılar az.")
        else:
            print("   ✅ Oran dengeli görünüyor.")
    
    # Trajectory boyutu analizi
    small_trajs = sum(1 for s in traj_sizes if s < 5)
    if small_trajs > len(traj_sizes) * 0.5:
        print(f"\n⚠️ UYARI: Trajectory'lerin %{100*small_trajs/len(traj_sizes):.0f}'i 5'ten az fingerprint içeriyor!")
        print("   → time_threshold parametresini artırın (örn: 600)")
    
    # Öneriler
    print(f"\n💡 İYİLEŞTİRME ÖNERİLERİ:")
    
    if len(traj_classes) > 0 and max(traj_classes.values()) / min(traj_classes.values()) > 3:
        print("   1. Veri dengesizliği var → Class weights kullanın")
    
    if np.mean(wifi_counts) < 50:
        print("   2. WiFi sinyali zayıf → Daha güçlü özellik çıkarımı kullanın")
    
    if len(elevations_df) < len(steps) / 10:
        print("   3. Elevation sayısı düşük → Katlar arası bağlantıları artırın")
    
    if np.mean(traj_sizes) < 10:
        print("   4. Trajectory'ler çok kısa → time_threshold'u artırın")
    
    print(f"\n📁 Oluşturulan dosyalar:")
    files = [
        f"{output_prefix}_fingerprints.json",
        f"{output_prefix}_lookup.json",
        f"{output_prefix}_steps.csv",
        f"{output_prefix}_elevations.csv",
        f"{output_prefix}_GT.json"
    ]
    for f in files:
        print(f"   ✓ {f}")
    
    print("\n✨ Dönüşüm tamamlandı!")
    print("=" * 70)
    
    return {
        'trajectories': len(trajectory_metadata),
        'fingerprints': len(fingerprints),
        'steps': len(steps),
        'elevations': len(elevations_df),
        'avg_traj_size': np.mean(traj_sizes),
        'avg_wifi_count': np.mean(wifi_counts)
    }


def find_csv_files(path=".", pattern="data.csv", contains=""):
    """Belirtilen path'de CSV dosyalarını bulur."""
    files = []
    for root, dirs, filenames in os.walk(path):
        for filename in filenames:
            if pattern in filename and contains in filename:
                files.append(os.path.join(root, filename))
    return files


def batch_convert(search_path=".", pattern="data.csv", 
                 time_threshold=300, use_space_id=False):
    """Birden fazla dosyayı toplu olarak dönüştürür."""
    files = find_csv_files(search_path, pattern)
    
    if not files:
        print(f"⚠️ '{pattern}' desenine uygun dosya bulunamadı!")
        return
    
    print(f"\n🔍 {len(files)} dosya bulundu:")
    for f in files:
        print(f"   • {f}")
    
    print("\n" + "="*70)
    
    results = []
    for i, file_path in enumerate(files, 1):
        print(f"\n[{i}/{len(files)}] İşleniyor: {file_path}")
        
        # Çıktı dizini oluştur
        output_dir = Path(file_path).stem
        os.makedirs(output_dir, exist_ok=True)
        
        # Dönüştür
        output_prefix = os.path.join(output_dir, Path(file_path).stem)
        
        try:
            stats = convert_uji_to_task2(file_path, output_prefix,
                                        time_threshold=time_threshold,
                                        use_space_id=use_space_id)
            results.append({
                'file': file_path,
                'success': True,
                'stats': stats
            })
        except Exception as e:
            print(f"❌ HATA: {e}")
            import traceback
            traceback.print_exc()
            results.append({
                'file': file_path,
                'success': False,
                'error': str(e)
            })
    
    # Özet rapor
    print("\n" + "="*70)
    print("📋 TOPLU DÖNÜŞÜM RAPORU")
    print("="*70)
    
    success_count = sum(1 for r in results if r['success'])
    print(f"\n✅ Başarılı: {success_count}/{len(results)}")
    
    if success_count > 0:
        total_trajs = sum(r['stats']['trajectories'] for r in results if r['success'])
        total_fps = sum(r['stats']['fingerprints'] for r in results if r['success'])
        avg_traj_size = np.mean([r['stats']['avg_traj_size'] for r in results if r['success']])
        
        print(f"\n📊 Toplam:")
        print(f"   • Trajectory: {total_trajs}")
        print(f"   • Fingerprint: {total_fps}")
        print(f"   • Ortalama Trajectory Boyutu: {avg_traj_size:.1f}")
    
    failed = [r for r in results if not r['success']]
    if failed:
        print(f"\n❌ Başarısız: {len(failed)}")
        for r in failed:
            print(f"   • {r['file']}: {r['error']}")



In [6]:
for i in files_add:
    temp=i[:-4].replace("\\","/")
    temp=temp
    convert_uji_to_task2(i, temp, time_threshold=600, use_space_id=False)

📍 UJIIndoorLoc -> Task2 Format Dönüşümü
📂 Girdi: ./ValidationGeo\data.csv
⏱️ Zaman Eşiği: 600 saniye
🏠 SPACEID Kullanımı: False

📊 Toplam fingerprint: 1111
📊 Bina sayısı: 3
📊 Kat sayısı: 5
📊 Kullanıcı sayısı: 1

📊 VERİ DAĞILIMI ANALİZİ:

🏢 Bina 0:
   Kat 0 (Class 0): 78 fingerprint
   Kat 1 (Class 1): 208 fingerprint
   Kat 2 (Class 2): 165 fingerprint
   Kat 3 (Class 3): 85 fingerprint

🏢 Bina 1:
   Kat 0 (Class 4): 30 fingerprint
   Kat 1 (Class 5): 143 fingerprint
   Kat 2 (Class 6): 87 fingerprint
   Kat 3 (Class 7): 47 fingerprint

🏢 Bina 2:
   Kat 0 (Class 8): 24 fingerprint
   Kat 1 (Class 9): 111 fingerprint
   Kat 2 (Class 10): 54 fingerprint
   Kat 3 (Class 11): 40 fingerprint
   Kat 4 (Class 12): 39 fingerprint

🎯 SINIF DAĞILIMI:
   Class 0: 78 fingerprint
   Class 1: 208 fingerprint
   Class 2: 165 fingerprint
   Class 3: 85 fingerprint
   Class 4: 30 fingerprint
   Class 5: 143 fingerprint
   Class 6: 87 fingerprint
   Class 7: 47 fingerprint
   Class 8: 24 fingerprint
   

In [5]:
i

'./TrainingWDE\\data.csv'